In [ ]:
"""
Created on Fri Aug 19 15:46 2022

This script is to double check Favier and Lipscomb RMSEs

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.plume_functions as pf
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.analysis_functions as anaf

import distributed

import matplotlib as mpl
import glob

In [ ]:
#home_path = '/Users/claraburgard/bettik_clara/'
home_path='/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/generic_plots/'

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM006/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].where(file_isf['isf_name'] != 'Ekstrom', np.array('Ekström', dtype=object))
isf_names = file_isf['isf_name']

In [ ]:
run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_list = []


ref_Gt_list = []


for n,nemo_run in enumerate(run_list):

    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

    ### READ IN THE REFERENCE
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt)
    
    ### READ IN THE PARAM FILES 
    
    # Param files
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    melt_param_files_favier = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_favier.nc')))
    melt_param_files_lipscomb = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_lipscomb*.nc')))
    melt_param_files = melt_param_files_favier + melt_param_files_lipscomb
    
    param_list = []
    for mfilename in melt_param_files:
        paramname = mfilename[91:201].split('.')[0].split('_')
        #paramname = mfilename[91:157].split('.')[0]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
        
    ds_melt_param  = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param.coords:        
        ds_melt_param = ds_melt_param.drop('param').drop('option')
    else:
        ds_melt_param = ds_melt_param.drop('param')
    ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
    ds_melt_param = ds_melt_param.assign_coords(param=param_list)
    diff_Gt = ds_melt_param['melt_1D_Gt_per_y'] - ref_Gt
    diff_Gt = diff_Gt.assign_coords({'time': np.arange(1,len(diff_Gt.time)+1)+n*50})
    diff_Gt_list.append(diff_Gt)

    
diff_Gt_all = xr.concat(diff_Gt_list, dim='time')
ref_Gt_all = xr.concat(ref_Gt_list, dim='nemo_run')
    

In [ ]:
diff_Gt_all.load()

In [ ]:
RMSE_Gt_all = np.sqrt((diff_Gt_all**2).mean(['time','Nisf']))


In [ ]:
for mparam in RMSE_Gt_all.param:
    print(mparam.values, RMSE_Gt_all.sel(param=mparam).values)

In [ ]:
run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_list = []


ref_Gt_list = []


for n,nemo_run in enumerate(run_list):

    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

    ### READ IN THE REFERENCE
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt)
    
    ### READ IN THE PARAM FILES - CV TIME
    
    # Param files
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    melt_param_files_orig1 = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_orig.nc')))
    melt_param_files_orig2 = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_orig_oneFRIS.nc')))
    melt_param_files = melt_param_files_orig1 + melt_param_files_orig2
    
    param_list = []
    for mfilename in melt_param_files:
        paramname = mfilename[91:201].split('.')[0].split('_')[:-2]
        #paramname = mfilename[91:157].split('.')[0]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
        
    ds_melt_param  = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param.coords:        
        ds_melt_param = ds_melt_param.drop('param').drop('option')
    elif 'param' in ds_melt_param.coords:
        ds_melt_param = ds_melt_param.drop('param')
    ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
    ds_melt_param = ds_melt_param.assign_coords(param=param_list)
    diff_Gt = ds_melt_param['melt_1D_Gt_per_y'] - ref_Gt
    diff_Gt = diff_Gt.assign_coords({'time': np.arange(1,len(diff_Gt.time)+1)+n*50})
    diff_Gt_list.append(diff_Gt)

    
diff_Gt_all = xr.concat(diff_Gt_list, dim='time')
ref_Gt_all = xr.concat(ref_Gt_list, dim='nemo_run')
    

In [ ]:
RMSE_Gt_all = np.sqrt((diff_Gt_all**2).mean(['time','Nisf']))


In [ ]:
for mparam in RMSE_Gt_all.param:
    print(mparam.values, RMSE_Gt_all.sel(param=mparam).values)